In [46]:
import os
os.chdir(r'C:\Users\krism\Desktop\semestr7\AiBD\lab2\lab2\ ')
print(os.getcwd())

C:\Users\krism\Desktop\semestr7\AiBD\lab2\lab2


Import

In [47]:
import pandas as pd
import os

df = pd.read_csv("original_data/billboard.csv", encoding='cp1250')
# df

Meta/data_reader

In [48]:
import datetime
import inflect
import datetime

_ie = inflect.engine()

class Mapping:
    date_0 = "year"
    week_label_x  = lambda numeral: "x{}.week".format(_ie.ordinal(numeral))
    week_label_yrcw = lambda yrcw: "yr{}_cw{}".format(*yrcw)
    
    to_copy = ["year", "artist.inverted", "track", "time", "genre", "date.entered", "date.peaked"]
    
    det_date = lambda entry: datetime.datetime.strptime(entry["date.entered"], "%Y-%m-%d")
    date2yrcw = lambda date: date.isocalendar()[0:2]
    week_shift = lambda date, no_weeks: date+datetime.timedelta(days=7*no_weeks)
    cw_count2yr = 53
        
    wlx_boundaries = (1, 77)
    wlx_numeration_shift = 1

Conversion

In [49]:
# import meta  # <- Mappign as m
import pandas as pd

dfi_weeks_x = [search_ix for search_ix in range(*Mapping.wlx_boundaries) 
             if Mapping.week_label_x(search_ix) in df.axes[1]]
dfi_maxaxes_week_x = [Mapping.week_label_yrcw((cwix//Mapping.cw_count2yr, cwix%Mapping.cw_count2yr))
                                           for cwix in range(max(dfi_weeks_x)+Mapping.cw_count2yr)]
dfi_axes ={column_name: {} for column_name in Mapping.to_copy + dfi_maxaxes_week_x}  # for order

dfi = pd.DataFrame(dfi_axes)

for enix, entry in df.iterrows():
    em_date = Mapping.det_date(entry)
    em_yrcw = Mapping.date2yrcw(em_date)
    nominals = {nominal: entry.loc[nominal] for nominal in Mapping.to_copy}
    
    weeks_yrcw = {}
    for xwix in dfi_weeks_x:
        week_label_x = Mapping.week_label_x(xwix)
        week_score = entry.loc[week_label_x]
        try:
            week_score = int(week_score)
        except ValueError:
            continue
            
        date_det = Mapping.week_shift(em_date, xwix)
        yrcw = Mapping.date2yrcw(date_det)
        rel_yrcw = (yrcw[0] - em_yrcw[0], yrcw[1])
        week_label_yrcw = Mapping.week_label_yrcw(rel_yrcw)
        weeks_yrcw[week_label_yrcw] = week_score
            
    entry_i = dict(nominals, **weeks_yrcw)
    dfi = dfi.append(entry_i, ignore_index=True)

dfi.dropna(how='all', axis=1, inplace=True)
# dfi

Export

In [51]:
dfi.to_csv("original_data/importable_data/billboard_imp.csv")